# Web scraping na CVM

- Inicialmente, raspando apenas dados cadastrais de Ações e FIIs.
- Base da CVM não tem coluna de ticker. Então, depois dar merge dela com a base manual através do CNPJ para obter setor/segmento dos ativos da base da CVM.
- Base manual de ações já tem setor/segmento, mas vou puxar da CVM também por 2 motivos: Mnter o padrão e também porque base da CVM tem várias outras informações que podem ser úteis no futuro.

In [11]:
import pandas as pd

# Ações

In [12]:
# URL do CSV
url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv"

# Lê diretamente
df_cvm_cad_cia_aberta = pd.read_csv(url, encoding='ISO-8859-1', sep=';')

print(f"Colunas: {df_cvm_cad_cia_aberta.columns.tolist()}")
print(f"Linhas: {len(df_cvm_cad_cia_aberta)}")

display(df_cvm_cad_cia_aberta.head())
# display(df_cvm_cad_cia_aberta[['SETOR_ATIV']].head())

Colunas: ['CNPJ_CIA', 'DENOM_SOCIAL', 'DENOM_COMERC', 'DT_REG', 'DT_CONST', 'DT_CANCEL', 'MOTIVO_CANCEL', 'SIT', 'DT_INI_SIT', 'CD_CVM', 'SETOR_ATIV', 'TP_MERC', 'CATEG_REG', 'DT_INI_CATEG', 'SIT_EMISSOR', 'DT_INI_SIT_EMISSOR', 'CONTROLE_ACIONARIO', 'TP_ENDER', 'LOGRADOURO', 'COMPL', 'BAIRRO', 'MUN', 'UF', 'PAIS', 'CEP', 'DDD_TEL', 'TEL', 'DDD_FAX', 'FAX', 'EMAIL', 'TP_RESP', 'RESP', 'DT_INI_RESP', 'LOGRADOURO_RESP', 'COMPL_RESP', 'BAIRRO_RESP', 'MUN_RESP', 'UF_RESP', 'PAIS_RESP', 'CEP_RESP', 'DDD_TEL_RESP', 'TEL_RESP', 'DDD_FAX_RESP', 'FAX_RESP', 'EMAIL_RESP', 'CNPJ_AUDITOR', 'AUDITOR']
Linhas: 2650


,CNPJ_CIA,DENOM_SOCIAL,DENOM_COMERC,DT_REG,DT_CONST,DT_CANCEL,MOTIVO_CANCEL,SIT,DT_INI_SIT,CD_CVM,...,UF_RESP,PAIS_RESP,CEP_RESP,DDD_TEL_RESP,TEL_RESP,DDD_FAX_RESP,FAX_RESP,EMAIL_RESP,CNPJ_AUDITOR,AUDITOR
0,08.773.135/0001-00,2W ECOBANK S.A. - EM RECUPERAÇÃO JUDICIAL,2W ECOBANK S.A.,2020-10-29,2007-03-23,NaN,NaN,ATIVO,2020-10-29,25224,...,NaN,NaN,4711905.0,11.0,39579400.0,NaN,NaN,juridico@2wecobank.com.br,10.830.108/0001-65,GRANT THORNTON AUDITORES INDEPENDENTES LTDA.
1,11.396.633/0001-87,3A COMPANHIA SECURITIZADORA,TRIPLO A COMPANHIA SECURITIZADORA,2010-03-08,2009-11-03,2015-12-18,Cancelamento Voluntário - IN CVM 480/09,CANCELADA,2015-12-18,21954,...,RJ,NaN,20020000.0,21.0,22338867.0,21.0,22338867.0,juridico@triploasec.com.br,60.525.706/0001-07,MOORE STEPHENS LIMA LUCCHESI AUDITORES INDEPEN...
2,01.547.749/0001-16,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,521 PARTICIPAÇÕES S/A,1997-07-11,1996-07-30,NaN,NaN,ATIVO,1997-07-11,16330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02.878.522/0001-16,IRKO HIRASHIMA AUDITORES INDEPENDENTES LTDA.
3,01.851.771/0001-55,524 PARTICIPAÇOES SA,524 PARTICIPACOES SA,1997-05-30,1997-04-02,NaN,NaN,ATIVO,1997-05-30,16284,...,RJ,NaN,20020010.0,21.0,38043700.0,NaN,NaN,gar@opportunity.com.br,10.830.108/0001-65,GRANT THORNTON AUDITORES INDEPENDENTES LTDA.
4,01.919.008/0001-19,525 PARTICIPAÇOES SA,525 PARTICIPACOES SA,1997-07-16,1997-04-02,2006-05-30,ELISÃO POR INCORPORAÇÃO,CANCELADA,2006-05-30,16349,...,RJ,NaN,22290160.0,21.0,21967200.0,21.0,21967201.0,kevin@mattosfilho.com.br,11.245.719/0001-09,DIRECTA AUDITORES


In [29]:
filtro = (df_cvm_cad_cia_aberta['CNPJ_CIA'] == '17.281.106/0001-03')

df_filtrado = df_cvm_cad_cia_aberta.loc[filtro].copy()

unicos = df_cvm_cad_cia_aberta['CONTROLE_ACIONARIO'].value_counts()#.drop_duplicates().reset_index()
unicos



CONTROLE_ACIONARIO
PRIVADO                2113
PRIVADO HOLDING         289
ESTRANGEIRO             111
ESTATAL                  88
ESTRANGEIRO HOLDING      35
ESTATAL HOLDING          12
Name: count, dtype: int64

# Fiis

In [14]:
import zipfile
from io import BytesIO
import requests

In [ ]:
# As 3 bases disponibilizadas nesse arquivo têm 1 linha para cada mês.
# Então, quando for usar para dados cadastrais, remover duplicadas pela col de CNPJ mantendo data + recente.
url_zip = "https://dados.cvm.gov.br/dados/FII/DOC/INF_MENSAL/DADOS/inf_mensal_fii_2025.zip"

response = requests.get(url_zip)
zip_bytes = BytesIO(response.content)

with zipfile.ZipFile(zip_bytes) as z:
    print("Arquivos no ZIP:", z.namelist())
    
    # Nome correto do arquivo (verifique o output acima)
    arquivo_csv = z.namelist()[0]  # Pega o 3o arquivo: 'inf_mensal_fii_geral_2025.csv'
    
    # Lê com configurações específicas para CVM
    df_inf_mensal_fii_geral_2025 = pd.read_csv(
        z.open(arquivo_csv),
        encoding='ISO-8859-1',
        sep=';',           # ⬅️ SEPARADOR CORRETO
        # skiprows=1,        # ⬅️ Pula linha de cabeçalho extra
        on_bad_lines='skip',  # ⬅️ Ignora linhas problemáticas
        dtype_backend='pyarrow'  # Mais eficiente
    )
    
print(f"✅ CSV carregado: {len(df_inf_mensal_fii_geral_2025)} linhas, {len(df_inf_mensal_fii_geral_2025.columns)} colunas")
print("Colunas:", df_inf_mensal_fii_geral_2025.columns.tolist())
print("\nPrimeiras linhas:")

df_inf_mensal_fii_geral_2025



Arquivos no ZIP: ['inf_mensal_fii_ativo_passivo_2025.csv', 'inf_mensal_fii_complemento_2025.csv', 'inf_mensal_fii_geral_2025.csv']
✅ CSV carregado: 12545 linhas, 52 colunas
Colunas: ['CNPJ_Fundo_Classe', 'Data_Referencia', 'Versao', 'Total_Necessidades_Liquidez', 'Disponibilidades', 'Titulos_Publicos', 'Titulos_Privados', 'Fundos_Renda_Fixa', 'Total_Investido', 'Direitos_Bens_Imoveis', 'Terrenos', 'Imoveis_Renda_Acabados', 'Imoveis_Renda_Construcao', 'Imoveis_Venda_Acabados', 'Imoveis_Venda_Construcao', 'Outros_Direitos_Reais', 'Acoes', 'Debentures', 'Bonus_Subscricao', 'Certificados_Deposito_Valores_Mobiliarios', 'Cedulas_Debentures', 'Fundo_Acoes', 'FIP', 'FII', 'FDIC', 'Outras_Cotas_FI', 'Notas_Promissorias', 'Acoes_Sociedades_Atividades_FII', 'Cotas_Sociedades_Atividades_FII', 'CEPAC', 'CRI', 'CRI_CRA', 'Letras_Hipotecarias', 'LCI', 'LCI_LCA', 'LIG', 'Outros_Valores_Mobliarios', 'Valores_Receber', 'Contas_Receber_Aluguel', 'Contas_Receber_Venda_Imoveis', 'Outros_Valores_Receber', '

,CNPJ_Fundo_Classe,Data_Referencia,Versao,Total_Necessidades_Liquidez,Disponibilidades,Titulos_Publicos,Titulos_Privados,Fundos_Renda_Fixa,Total_Investido,Direitos_Bens_Imoveis,...,Taxa_Administracao_Pagar,Taxa_Performance_Pagar,Obrigacoes_Aquisicao_Imoveis,Adiantamento_Venda_Imoveis,Adiantamento_Alugueis,Obrigacoes_Securitizacao_Recebiveis,Instrumentos_Financeiros_Derivativos,Provisoes_Contigencias,Outros_Valores_Pagar,Total_Passivo
0,00.332.266/0001-31,2025-01-01,1,5819997.69,433459.39,0.0,0.0,5386538.3,480711200.0,480711200.0,...,108233.38,0.0,0.0,0.0,650002.6,0.0,0.0,0.0,4482600.45,6681378.29
1,00.332.266/0001-31,2025-02-01,1,5802393.24,425428.83,0.0,0.0,5376964.41,480711200.0,480711200.0,...,108533.38,0.0,0.0,0.0,650002.6,0.0,0.0,0.0,4059245.29,6258393.13
2,00.332.266/0001-31,2025-03-01,1,5757991.5,609375.92,0.0,0.0,5148615.58,480711200.0,480711200.0,...,108533.37,0.0,0.0,0.0,650002.6,0.0,0.0,0.0,3724155.38,5923373.21
3,00.332.266/0001-31,2025-04-01,1,7172252.39,3605034.86,0.0,0.0,3567217.53,480724700.0,480724700.0,...,108533.38,0.0,0.0,0.0,650002.6,0.0,0.0,0.0,3895151.76,6094439.6
4,00.332.266/0001-31,2025-05-01,1,7028967.98,2995920.4,0.0,0.0,4033047.58,480724700.0,480724700.0,...,108533.38,0.0,0.0,0.0,650002.6,0.0,0.0,0.0,3085621.29,6685053.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12540,97.521.225/0001-25,2025-07-01,1,110444302.66,0.0,0.0,0.0,110444302.66,4032252382.32,11937060.0,...,3095720.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,709428.95,47537678.77
12541,97.521.225/0001-25,2025-08-01,1,99420504.58,0.0,0.0,0.0,99420504.58,4075134311.45,9147060.0,...,3088182.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264716.43,47085428.5
12542,97.521.225/0001-25,2025-09-01,1,47228885.78,215775.45,0.0,0.0,47013110.33,4086679900.93,9147060.0,...,3161001.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,233460.04,47126991.03
12543,97.521.225/0001-25,2025-10-01,1,64994504.71,0.0,0.0,0.0,64994504.71,4082712252.63,9147060.0,...,3130658.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9390427.94,55859751.33


In [16]:
df_inf_mensal_fii_geral_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12545 entries, 0 to 12544
Data columns (total 52 columns):
 #   Column                                     Non-Null Count  Dtype          
---  ------                                     --------------  -----          
 0   CNPJ_Fundo_Classe                          12545 non-null  string[pyarrow]
 1   Data_Referencia                            12545 non-null  string[pyarrow]
 2   Versao                                     12545 non-null  int64[pyarrow] 
 3   Total_Necessidades_Liquidez                12545 non-null  double[pyarrow]
 4   Disponibilidades                           12527 non-null  double[pyarrow]
 5   Titulos_Publicos                           10969 non-null  double[pyarrow]
 6   Titulos_Privados                           10665 non-null  double[pyarrow]
 7   Fundos_Renda_Fixa                          12177 non-null  double[pyarrow]
 8   Total_Investido                            12545 non-null  double[pyarrow]
 9   Direit

In [1]:
def carregar_dados_inf_mensal_fii(url_zip):
    """
    Carrega os 3 arquivos CSV do ZIP da CVM e retorna DataFrames separados.
    """
    
    import requests
    import pandas as pd
    import zipfile
    from io import BytesIO
    
    # Baixa o arquivo ZIP
    response = requests.get(url_zip)
    zip_bytes = BytesIO(response.content)
    
    # Lista para armazenar os DataFrames
    dataframes = []
    
    with zipfile.ZipFile(zip_bytes) as z:
        # Pega a lista de todos os arquivos no ZIP
        arquivos = z.namelist()
        print(f"Arquivos encontrados no ZIP: {arquivos}")
        
        # Verifica se há pelo menos 3 arquivos
        if len(arquivos) < 3:
            raise ValueError(f"Esperados 3 arquivos no ZIP, encontrados {len(arquivos)}")
        
        # Itera sobre os 3 primeiros arquivos
        for i, arquivo_csv in enumerate(arquivos[:3]):
            print(f"\n📂 Processando arquivo {i+1}: {arquivo_csv}")
            
            # Lê cada arquivo CSV com configurações específicas para CVM
            df = pd.read_csv(
                z.open(arquivo_csv),
                encoding='ISO-8859-1',
                sep=';',               # Separador correto para dados CVM
                on_bad_lines='skip',   # Ignora linhas problemáticas
                dtype_backend='pyarrow' # Mais eficiente em memória
            )
            
            print(f"✅ CSV carregado: {len(df)} linhas, {len(df.columns)} colunas")
            dataframes.append(df)
    
    # Atribui os DataFrames aos nomes específicos
    df_inf_mensal_fii_ativo_passivo = dataframes[0]  # Índice 0
    df_inf_mensal_fii_complemento = dataframes[1]    # Índice 1  
    df_inf_mensal_fii_geral = dataframes[2]          # Índice 2
    
    print("\n" + "="*50)
    print("✅ TODOS OS ARQUIVOS CARREGADOS COM SUCESSO")
    print(f"1. Ativo/Passivo: {len(df_inf_mensal_fii_ativo_passivo)} linhas")
    print(f"2. Complemento: {len(df_inf_mensal_fii_complemento)} linhas")
    print(f"3. Geral: {len(df_inf_mensal_fii_geral)} linhas")
    
    # Retorna os 3 DataFrames como tupla
    return (
        df_inf_mensal_fii_ativo_passivo,
        df_inf_mensal_fii_complemento, 
        df_inf_mensal_fii_geral
    )


# Exemplo de uso:
url_zip = "https://dados.cvm.gov.br/dados/FII/DOC/INF_MENSAL/DADOS/inf_mensal_fii_2025.zip"

# Chama a função e recebe os 3 DataFrames
(df_ativo_passivo, df_complemento, df_geral) = carregar_dados_inf_mensal_fii(url_zip)

# Agora você tem os 3 DataFrames disponíveis:
# df_ativo_passivo → df_inf_mensal_fii_ativo_passivo
# df_complemento → df_inf_mensal_fii_complemento  
# df_geral → df_inf_mensal_fii_geral

Arquivos encontrados no ZIP: ['inf_mensal_fii_ativo_passivo_2025.csv', 'inf_mensal_fii_complemento_2025.csv', 'inf_mensal_fii_geral_2025.csv']

📂 Processando arquivo 1: inf_mensal_fii_ativo_passivo_2025.csv
✅ CSV carregado: 13651 linhas, 52 colunas

📂 Processando arquivo 2: inf_mensal_fii_complemento_2025.csv
✅ CSV carregado: 13651 linhas, 30 colunas

📂 Processando arquivo 3: inf_mensal_fii_geral_2025.csv
✅ CSV carregado: 13651 linhas, 37 colunas

✅ TODOS OS ARQUIVOS CARREGADOS COM SUCESSO
1. Ativo/Passivo: 13651 linhas
2. Complemento: 13651 linhas
3. Geral: 13651 linhas


In [2]:
df_inf_mensal_fii_ativo_passivo
df_inf_mensal_fii_complemento
df_inf_mensal_fii_geral

NameError: name 'df_inf_mensal_fii_ativo_passivo' is not defined